In [19]:
import psycopg2
import pandas as pd

In [20]:
# Replace these parameters with your actual database credentials
db_user="postgres"
db_name = "hck028"
db_password = "postgres" #Use your own password
db_host = "localhost"  # Usually "localhost" if running locally
db_port = "5432"  # Default is 5432



In [21]:
connection = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

In [22]:
cursor = connection.cursor() #membuat koneksi ke sql

In [23]:
query = '''
SELECT 
	conference,
       SUM(CASE WHEN year = 'FR' THEN total_player ELSE 0 END) fr,
       SUM(CASE WHEN year = 'SO' THEN total_player ELSE 0 END) so,
       SUM(CASE WHEN year = 'JR' THEN total_player ELSE 0 END) jr,
       SUM(CASE WHEN year = 'SR' THEN total_player ELSE 0 END) sr
FROM (
	SELECT
		t.conference,
		p.year,
		COUNT(1) as total_player
	FROM 
		players p
		JOIN teams t
		ON p.school_name = t.school_name
	GROUP BY
		1, 2
	ORDER BY
		1, 2
) babs
GROUP BY
	1 
ORDER BY
	1
'''
cursor.execute(query)
result = cursor.fetchall()
print(result)

[('ACC', Decimal('607'), Decimal('341'), Decimal('356'), Decimal('259')), ('American Athletic', Decimal('418'), Decimal('247'), Decimal('241'), Decimal('218')), ('Big 12', Decimal('456'), Decimal('254'), Decimal('270'), Decimal('210')), ('Big Ten', Decimal('636'), Decimal('314'), Decimal('284'), Decimal('232')), ('Conference USA', Decimal('519'), Decimal('324'), Decimal('351'), Decimal('301')), ('FBS Independents', Decimal('279'), Decimal('186'), Decimal('191'), Decimal('171')), ('Mid-American', Decimal('551'), Decimal('276'), Decimal('236'), Decimal('329')), ('Mountain West', Decimal('338'), Decimal('201'), Decimal('221'), Decimal('183'))]


In [24]:
df = pd.read_sql_query(query, connection) # import query ke python through pandas

df

/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_24231/3233301777.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection) # import query ke python through pandas


,conference,fr,so,jr,sr
0,ACC,607.0,341.0,356.0,259.0
1,American Athletic,418.0,247.0,241.0,218.0
2,Big 12,456.0,254.0,270.0,210.0
3,Big Ten,636.0,314.0,284.0,232.0
4,Conference USA,519.0,324.0,351.0,301.0
5,FBS Independents,279.0,186.0,191.0,171.0
6,Mid-American,551.0,276.0,236.0,329.0
7,Mountain West,338.0,201.0,221.0,183.0


In [25]:
df.to_csv('pivot_result.csv', index=False)

In [27]:
# Create new Table in Python

q = '''
    CREATE TABLE IF NOT EXISTS students (
        id SERIAL PRIMARY KEY,
        name VARCHAR(50),
        age INTEGER,
        campus_id INTEGER,
        total_grade FLOAT
        );
    '''

cursor.execute(q)
connection.commit()

In [28]:
cursor.close()
connection.close()